# 4.1 微博红酒数据预处理

In [1]:
import csv
import json
csvfile = open('./data/微博红酒.csv', 'r')
reader = csv.reader(csvfile)#读取到的数据是将每行数据当做列表返回的
rows = []#用来存储解析后的没条数据
for row in reader:
    row_str = ",".join(row)#row为list类型需转为str，该数据变为字典型字符串
    row_dict = json.loads(row_str)
        
    #将每行数据中嵌套字典拆开存储到列表中
    newdict = {}
    for k in row_dict:
        if type(row_dict[k]) == str:#将键值对赋给新字典
            newdict[k] = row_dict[k]
        elif type(row_dict[k]) == dict:#若存在嵌套字典，将该字典中的key和value作为属性和属性值
            newdict.update(row_dict[k])
    rows.append(newdict)

In [2]:
len(rows)

24611

In [3]:
rows[2]

{'id': '10099536052',
 'url': 'https://weibo.com/1823411197/ghE0Li',
 'keyword': '莱茵黑森 红酒',
 'post_time': '2019-05-15 14:25:15',
 'post_text': '今晚有红衫鱼  蚝仔烙 粉丝带子，如果把这支红酒换成加拿大冰酒或者99年莱茵黑森白葡就完美啦哈哈哈哈超有满足感的一餐，脂肪来吧来吧 ',
 'post_source': '彩信',
 'description': '还是很想你……dbx',
 'gender': 'f',
 'name': '舞夜eva',
 '昵称': '舞夜eva',
 '所在地': '广东 广州',
 '性别': '女',
 '生日': '11月4日',
 '博客': '',
 '个性域名': 'https://weibo.com/evasdream',
 '简介': '还是很想你……dbx',
 '注册时间': '2010-09-25',
 '公司': '中外|职位：设计部',
 '标签': '小蝎子|设计师|化妆师|大胃王|小肉丸'}

In [4]:
import pandas as pd
df = pd.DataFrame(rows)#将字典型数组转为DataFrame形式
print(df.shape)
print(df.columns)

(24611, 32)
Index(['', 'MSN', 'QQ', 'description', 'gender', 'id', 'keyword', 'name',
       'post_source', 'post_text', 'post_time', 'url', '个性域名', '中专技校', '公司',
       '初中', '博客', '大学', '小学', '性别', '性取向', '感情状况', '所在地', '昵称', '标签', '注册时间',
       '生日', '真实姓名', '简介', '血型', '邮箱', '高中'],
      dtype='object')


In [5]:
df = df[["keyword","post_time","所在地","性别","生日","gender"]]
print(df.shape)
print(df.columns)
#将DataFrame 写入到 csv 文件
df.to_csv("wine_df_weibo.csv",encoding="utf-8_sig", index = False)

(24611, 6)
Index(['keyword', 'post_time', '所在地', '性别', '生日', 'gender'], dtype='object')


In [6]:
# 处理红酒名称（统一规范中文/英文的格式）
#去keyword中“红酒”字符
df['keyword']=df['keyword'].str.split('红酒').str[0]

#整理红酒品牌
brand = pd.read_csv("./data/红酒品牌.csv",header=None,encoding="utf-8")
brands =[]
for k1,k2 in zip(list(brand[0]),list(brand[1])):
    if pd.isnull(k2):
        brands.append(k1)
    else:
        brands.append(k1+"/"+k2)
        
# 品牌替换
def modify_keywords(w, lists):
    for b in lists:
        if w.strip() in b:
            return b
    return w.strip()


df['keyword'] = df['keyword'].apply(modify_keywords, lists =brands)
df['keyword']=df['keyword'].str.lower()#转化为小写

print(df["keyword"].head(10))

0               卓林/zonin
1    怡园酒庄/grace vineyard
2                   莱茵黑森
3          长城/greatwall 
4                     奥兰
5             华东/huadong
6       黄尾袋鼠/yellow tail
7       黄尾袋鼠/yellow tail
8                     奥兰
9                    安徒生
Name: keyword, dtype: object


In [7]:
df.head(2)

,keyword,post_time,所在地,性别,生日,gender
0,卓林/zonin,2019-05-15 14:29:29,NaN,NaN,NaN,m
1,怡园酒庄/grace vineyard,2019-05-15 14:25:30,香港 其他,女,NaN,f


In [8]:
# 处理性别
# df.count()
# df.groupby("gender").size().sort_values(ascending=False)
df.drop("性别",axis=1,inplace=True)

In [9]:
# 处理年龄
# df.count()
def age(x):
    index = str(x).find('年')
    if index == -1 :
        return 9999
    else:
        res = 2019-int(str(x)[:index])
        # 超过100岁，视为不合理
        if res>100:
            return 9999
        else:
            return 2019-int(str(x)[:index])

df['age'] = df['生日'].map(age)
df.drop("生日",axis=1,inplace=True)

In [10]:
df.head(20)

,keyword,post_time,所在地,gender,age
0,卓林/zonin,2019-05-15 14:29:29,NaN,m,9999
1,怡园酒庄/grace vineyard,2019-05-15 14:25:30,香港 其他,f,9999
2,莱茵黑森,2019-05-15 14:25:15,广东 广州,f,9999
3,长城/greatwall,2019-05-15 14:24:20,上海 浦东新区,m,34
4,奥兰,2019-05-15 14:15:26,北京,f,9999
5,华东/huadong,2019-05-15 14:27:43,山东 青岛,f,9999
6,黄尾袋鼠/yellow tail,2019-05-15 14:25:05,辽宁 大连,m,9999
7,黄尾袋鼠/yellow tail,2019-05-15 14:25:05,上海,m,9999
8,奥兰,2019-05-15 14:15:26,广东 深圳,f,9999
9,安徒生,2019-05-15 14:27:42,NaN,m,9999


In [11]:
# 处理地区
df['所在地'].fillna("其他",inplace=True)

def place(x):
    words = str(x).split(" ")
    if len(words) > 1:
        return words[0].strip(), words[1].strip()
    else:
        return words[0],"其他"
    
df["所在地"] = df["所在地"].map(place)
df['country'] = df["所在地"].apply(lambda x : x[0])
df['region'] = df['所在地'].apply(lambda x : x[1])
df.drop("所在地",axis = 1,inplace = True)

In [12]:
df.head(10)

,keyword,post_time,gender,age,country,region
0,卓林/zonin,2019-05-15 14:29:29,m,9999,其他,其他
1,怡园酒庄/grace vineyard,2019-05-15 14:25:30,f,9999,香港,其他
2,莱茵黑森,2019-05-15 14:25:15,f,9999,广东,广州
3,长城/greatwall,2019-05-15 14:24:20,m,34,上海,浦东新区
4,奥兰,2019-05-15 14:15:26,f,9999,北京,其他
5,华东/huadong,2019-05-15 14:27:43,f,9999,山东,青岛
6,黄尾袋鼠/yellow tail,2019-05-15 14:25:05,m,9999,辽宁,大连
7,黄尾袋鼠/yellow tail,2019-05-15 14:25:05,m,9999,上海,其他
8,奥兰,2019-05-15 14:15:26,f,9999,广东,深圳
9,安徒生,2019-05-15 14:27:42,m,9999,其他,其他


In [13]:
# 处理发布日期
df.drop("post_time",axis=1,inplace=True)

In [14]:
df.head(2)

,keyword,gender,age,country,region
0,卓林/zonin,m,9999,其他,其他
1,怡园酒庄/grace vineyard,f,9999,香港,其他


# 4.2 聚类分析

In [15]:
import pandas as pd
winedf = pd.read_csv("./prepare_for_weibo.csv")

In [16]:
winedf.dtypes

keyword                     object
price                      float64
产区                          object
原产地                         object
口感                          object
国产/进口                       object
特性                          object
甜度                          object
颜色                          object
冰酒/贵腐/甜酒                     int64
白葡萄酒                         int64
果味葡萄酒                        int64
桃红葡萄酒                        int64
红葡萄酒                         int64
起泡酒/香槟                       int64
马尔贝克（Malbec）                 int64
其它                           int64
梅洛（Merlot）                   int64
仙粉黛（Zinfandel）               int64
赤霞珠（Cabernet Sauvignon）      int64
西拉/设拉子（Syrah/Shiraz）         int64
雷司令（Riesling）                int64
桑娇维塞（Sangiovese）             int64
黑皮诺（Pinot Noir）              int64
长相思（Sauvignon Blanc）         int64
内比奥罗（Nebbiolo）               int64
佳美（Gamay）                    int64
品丽珠（Cabernet Franc）          int64
匹诺塔吉（Pinotage）      

In [17]:
resdf = pd.get_dummies(winedf) #非数值型都转化为one-hot

In [18]:
from sklearn.cluster import KMeans

kmodel = KMeans(n_clusters = 6,n_jobs=5)
kmodel.fit(resdf)        #训练模型
rs = kmodel.predict(resdf) #类别索引
winedf['label'] = rs

In [19]:
winedf.groupby("label").agg({'price':['min','max','count']})

price               
            min      max count
label                         
0          9.83   3399.0  3205
1          4.83  50000.0  4263
2          3.32  23040.0  3112
3         16.00   8800.0   206
4      78000.00  79560.0     2
5       3499.00  29800.0   163

In [20]:
classA = winedf[winedf.label==0].groupby(["keyword"]).size().sort_values(ascending=False).reset_index()
classA.columns=["keyword","A"]
classB = winedf[winedf.label==1].groupby(["keyword"]).size().sort_values(ascending=False).reset_index()
classB.columns=["keyword","B"]
classC = winedf[winedf.label==2].groupby(["keyword"]).size().sort_values(ascending=False).reset_index()
classC.columns=["keyword","C"]
classD = winedf[winedf.label==3].groupby(["keyword"]).size().sort_values(ascending=False).reset_index()
classD.columns=["keyword","D"]
classE = winedf[winedf.label==4].groupby(["keyword"]).size().sort_values(ascending=False).reset_index()
classE.columns=["keyword","E"]
classF = winedf[winedf.label==5].groupby(["keyword"]).size().sort_values(ascending=False).reset_index()
classF.columns=["keyword","F"]

In [21]:
print(classA.shape)
print(classB.shape)
print(classC.shape)
print(classD.shape)
print(classE.shape)
print(classF.shape)

(120, 2)
(172, 2)
(136, 2)
(56, 2)
(1, 2)
(11, 2)


In [22]:
classC[:10]

,keyword,C
0,长城/greatwall,606
1,张裕/changyu,219
2,通化/tonghua,180
3,王朝/dynasty,102
4,拉菲/lafite,98
5,黄尾袋鼠/yellow tail,97
6,杰卡斯/jacob’s creek,97
7,罗曼尼康帝庄园,96
8,高斯达,89
9,奔富/penfolds,88


In [23]:
level = pd.DataFrame(list(set(winedf['keyword'])),columns=["keyword"])
level = pd.merge(level,classA,how='left',on=["keyword"])
level = pd.merge(level,classB,how='left',on=["keyword"])
level = pd.merge(level,classC,how='left',on=["keyword"])
level = pd.merge(level,classD,how='left',on=["keyword"])
level = pd.merge(level,classF,how='left',on=["keyword"])
level = level.fillna(0)

level['max_value']=level.max(axis=1)
level=level[level.max_value>0]
print(level.shape)
def appendlevel(sr):
    levels=list('ABCDF')
    for i in levels:
        if sr[i] == sr['max_value']:
            return i
level['level'] = level.apply(lambda x:appendlevel(x),axis=1)#每一行apply

(222, 7)


In [24]:
level.groupby('level').size()

level
A     54
B    109
C     51
D      5
F      3
dtype: int64

In [25]:
tmp = level[level.level=='F'].sort_values("max_value",ascending=False)
list(tmp[tmp.max_value>=0]['keyword'])

['木桐古堡/ch. mouton rothschild', '拉图酒庄', '作品一号/opus one']

In [26]:
result = pd.merge(df,level[["keyword","level"]],how="left",on="keyword")

In [27]:
# 删除level为空的数据
result.dropna(axis='index',inplace=True)
print(result.shape)

(22524, 6)


In [28]:
result.to_csv("level.csv",encoding="utf-8_sig",index = False)